In [1]:
# Import libraries
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

# Define paths
train_data_dir = "dataset/train"
test_data_dir = "dataset/test"
img_width, img_height = 150, 150  # Adjust image size as needed
batch_size = 32  # Adjust batch size as needed
epochs = 20  # Adjust number of training epochs as needed

# Data augmentation (optional)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for testing

# Train data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Binary classification for tomato/orange
)

# Validation data generator (split a portion of training data for validation)
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Use part of training data for validation
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    validation_split=0.2  # 20% of training data for validation
)

# Test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,  # Test on single image at a time
    class_mode='binary',
    shuffle=False  # Maintain order for test images
)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping (optional)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)  # Stop training if validation loss doesn't improve for 3 epochs

# Train the model
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Make predictions on test images (modify to iterate through all test images)
predictions = model.predict_classes(test_generator)

# Print predictions (modify to print individual predictions for each test image)
print('Predicted labels:', predictions)


Found 1192 images belonging to 2 classes.


TypeError: ImageDataGenerator.flow_from_directory() got an unexpected keyword argument 'validation_split'